In [ ]:
# import locale

# print(locale.getpreferredencoding())


In [ ]:
!nvidia-smi -L
import psutil
print(psutil.virtual_memory())
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-f5e874ed-d98a-caff-74d4-c25808413113)
svmem(total=89639665664, available=88039993344, percent=1.8, used=791904256, free=85528317952, active=484843520, inactive=3234336768, buffers=279719936, cached=3039723520, shared=1372160, slab=156463104)
Mounted at /content/gdrive


In [ ]:
# Stopped mid level = 2? use continue
# Stopped mid level = 1 or 0? use upsample

lemode = ''                        # 'ancestral','primed','continue','cutcontinue','upsample'
lemodel = ''                            # CUSTOM-MODEL-NAME or '1b_lyrics'

leartist = "unknown"
legenre = "unknown"

lecount = 3
lesample_length_in_seconds = 330
lesampling_temperature = .98616616
lehop = [1,1,0.0625]                        #default [.5,.5,.125], optimized [1,1,0.0625]

lepath = '/content/gdrive/MyDrive/..'  #replace with path to output folder on Google Drive

leprompt_length_in_seconds= 30

lecut = 70                                  # used only on cutcontinue

leexportlyrics = False
leprogress = False # leave false to prevent issue with time stuff that crashes the program
leautorename = True

lemax_batch_size = 3
lelower_batch_size = 3
lechunk_size = 16
lelower_level_chunk_size = 32

lemulti = False

# Single

In [ ]:
lelyrics = """ """
leaudio_fileS = '/content/gdrive/MyDrive/...'   #if priming replace with path to the primer sample folder on Google Drive

# Multi

In [ ]:
leaudio_fileM = ('/content/gdrive/MyDrive/....')    #0  if priming with multiple files replace with path to the primer sample folder on Google Drive

lelyrics0 = """ """


# 1 😂 Sample 😂


In [ ]:
if lemode=='ancestral':
  leprompt_length_in_seconds=None
  leaudio_file = None
###############################################################################
###############################################################################

codes_file=None

#!pip install git+https://github.com/openai/jukebox.git

#@title Install Jukebox
#!pip install -q --upgrade git+https://github.com/craftmine1000/jukebox-saveopt.git

!git clone https://github.com/craftmine1000/jukebox-saveopt.git
!sed -i 's/==/>=/g' jukebox-saveopt/requirements.txt
!pip install jukebox-saveopt/

##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$#### autosave start
import os
from glob import glob

filex = "/usr/local/lib/python3.9/dist-packages/jukebox/sample.py"
fin = open(filex, "rt")
data = fin.read()
fin.close()

newtext = '''import fire
import os
from glob import glob

from termcolor import colored
from datetime import datetime

newtosample = True'''
data = data.replace('import fire',newtext)

newtext = '''starts = get_starts(total_length, prior.n_ctx, hop_length)
        counterr = 0
        x = None
        for start in starts:'''
data = data.replace('for start in get_starts(total_length, prior.n_ctx, hop_length):',newtext)

newtext = '''global newtosample
    newtosample = (new_tokens > 0)
    if new_tokens <= 0:'''
data = data.replace('if new_tokens <= 0:',newtext)

newtext = '''counterr += 1
            datea = datetime.now()
            zs = sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)
            if newtosample and counterr < len(starts):
                del x; x = None; prior.cpu(); empty_cache()
                x = prior.decode(zs[level:], start_level=level, bs_chunks=zs[level].shape[0])
                logdir = f"{hps.name}/level_{level}"
                if not os.path.exists(logdir):
                    os.makedirs(logdir)
                t.save(dict(zs=zs, labels=labels, sampling_kwargs=sampling_kwargs, x=x), f"{logdir}/data.pth.tar")
                save_wav(logdir, x, hps.sr)
                del x; prior.cuda(); empty_cache(); x = None
            dateb = datetime.now()
            timex = ((dateb-datea).total_seconds()/60.0)*(len(starts)-counterr)
            print(f"Step " + colored(counterr,'blue') + "/" + colored( len(starts),'red') + " ~ New to Sample: " + str(newtosample) + " ~ estimated remaining minutes: " + (colored('???','yellow'), colored(timex,'magenta'))[counterr > 1 and newtosample])'''
data = data.replace('zs = sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)',newtext)


newtext = """lepath=hps.name
        if level==2:
          for filex in glob(os.path.join(lepath + '/level_2','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-'))
        if level==1:
          for filex in glob(os.path.join(lepath + '/level_1','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-L1-'))
        if level==0:
          for filex in glob(os.path.join(lepath + '/level_0','item_*.wav')):
            os.rename(filex,filex.replace('item_',lepath.split('/')[-1] + '-L0-'))
        save_html("""
if leautorename:
  data = data.replace('save_html(',newtext)

if leexportlyrics == False:
  data = data.replace('if alignments is None','#if alignments is None')
  data = data.replace('alignments = get_alignment','#alignments = get_alignment')
  data = data.replace('save_html(','#save_html(')

if leprogress == False:
  data = data.replace('print(f"Step " +','#print(f"Step " +')

fin = open(filex, "wt")
fin.write(data)
fin.close()
##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$#### autosave end

###CUSTOM MODEL############ To Sample with Google Drive
if not '1b' in lemodel and not '5b' in lemodel:
  lemodelpath = '/content/gdrive/MyDrive/The_Well_Trained_Algorithm_Step3/juke/' + lemodel + '_prior/checkpoint_latest.pth.tar'
  fin = open("/usr/local/lib/python3.9/dist-packages/jukebox/hparams.py", "rt")
  data = fin.read();  fin.close()


  data += lemodel + """_prior = Hyperparams()
""" + lemodel + """_prior.update(prior_1b_lyrics)
""" + lemodel + """_prior.restore_prior='""" + lemodelpath + """'
""" + lemodel + """_prior.level=2
HPARAMS_REGISTRY['""" + lemodel + """_prior'] = """ + lemodel + """_prior
"""


  #data = data.replace('y_bins=(10,100)','y_bins=(604, 7898)')
  data = data.replace('min_duration=60.0','min_duration=24.0')
  data = data.replace('max_duration=600.0','max_duration=666.0')

  fin = open("/usr/local/lib/python3.9/dist-packages/jukebox/hparams.py", "wt")
  fin.write(data);  fin.close()

  fin = open("/usr/local/lib/python3.9/dist-packages/jukebox/make_models.py", "rt")
  data = fin.read(); fin.close()

  data = data.replace("#'your_model': ","'" +lemodel + "_model': ")
  data = data.replace('("you_vqvae_here", "your_upsampler_here", ..., "you_top_level_prior_here")','("vqvae",  "upsampler_level_0", "upsampler_level_1", "' + lemodel + '_prior"),')

  data = data.replace("dist.barrier()","dist.barrier(); print('1')")
  data = data.replace("checkpoint = t.load(restore, map_location=t.device('cpu'))","checkpoint = t.load(restore, map_location=t.device('cuda')); print('2')")

  fin = open("/usr/local/lib/python3.9/dist-packages/jukebox/make_models.py", "wt")
  fin.write(data); fin.close()

  lemodel = lemodel + "_model"
###CUSTOM MODEL END############

import jukebox
import torch as t
import librosa
import os

from datetime import datetime

from IPython.display import Audio
from jukebox.make_models import make_vqvae, make_prior, MODELS, make_model
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.sample import sample_single_window, _sample, \
                           sample_partial_window, upsample, \
                           load_prompts
from jukebox.utils.dist_utils import setup_dist_from_mpi
from jukebox.utils.torch_utils import empty_cache
rank, local_rank, device = setup_dist_from_mpi()

print(datetime.now().strftime("%H:%M:%S"))

model = lemodel
hps = Hyperparams()
hps.sr = 44100
hps.n_samples = lecount
hps.name = lepath

chunk_size = lechunk_size
max_batch_size = lemax_batch_size

hps.levels = 3
hps.hop_fraction = lehop

vqvae, *priors = MODELS[model]
vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length = 786432)), device)
top_prior = make_prior(setup_hparams(priors[-1], dict()), vqvae, device)

# Prime song creation using an arbitrary audio sample.
mode = lemode
codes_file=None
audio_file = (leaudio_fileS,leaudio_fileM)[lemulti==True]
prompt_length_in_seconds=leprompt_length_in_seconds


if os.path.exists(hps.name):
  # Identify the lowest level generated and continue from there.
  for level in [0, 1, 2]:
    data = f"{hps.name}/level_{level}/data.pth.tar"
    if os.path.isfile(data):
      mode = mode if 'continue' in mode else 'upsample'
      codes_file = data
      print(mode + 'ing from level ' + str(level))
      break
print('mode is now '+mode)

sample_hps = Hyperparams(dict(mode=mode, codes_file=codes_file, audio_file=audio_file, prompt_length_in_seconds=prompt_length_in_seconds))

sample_length_in_seconds = lesample_length_in_seconds
hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
assert hps.sample_length >= top_prior.n_ctx*top_prior.raw_to_tokens, f'Please choose a larger sampling rate'

if lemulti==True:
  metas = [dict(artist = leartist, genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics0, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics1, ), ]    + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics2, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics3, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics4, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics5, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics6, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics7, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics8, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics9, ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics10 ), ]   + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics11, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics12, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics13, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics14, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics15, ), ]  + [dict(artist = leartist,  genre = legenre, total_length = hps.sample_length, offset = 0, lyrics = lelyrics16, ), ]
else:
  metas = [dict(artist = leartist,
              genre = legenre,
              total_length = hps.sample_length,
              offset = 0,
              lyrics = lelyrics,
              ),
            ] * hps.n_samples
labels = [None, None, top_prior.labeller.get_batch_labels(metas, 'cuda')]


#----------------------------------------------------------2

sampling_temperature = lesampling_temperature
lower_batch_size = lelower_batch_size
max_batch_size = lemax_batch_size
lower_level_chunk_size = lelower_level_chunk_size
chunk_size = lechunk_size
sampling_kwargs = [dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                        chunk_size=lower_level_chunk_size),
                    dict(temp=.99, fp16=True, max_batch_size=lower_batch_size,
                         chunk_size=lower_level_chunk_size),
                    dict(temp=sampling_temperature, fp16=True,
                         max_batch_size=max_batch_size, chunk_size=chunk_size)]

print(sample_hps.mode)
print(sample_hps.prompt_length_in_seconds)
print(hps.sr)
print(top_prior.raw_to_tokens)
print('aaaaaaaaaaaaaaaaaaaaaaaaaaaa 4.55')

if sample_hps.mode == 'ancestral':
  zs = [t.zeros(hps.n_samples,0,dtype=t.long, device='cuda') for _ in range(len(priors))]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'upsample':
  assert sample_hps.codes_file is not None
  # Load codes.
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  del data
  print('Falling through to the upsample step later in the notebook.')
elif sample_hps.mode == 'primed':
  assert sample_hps.audio_file is not None
  audio_files = sample_hps.audio_file.split(',')
  duration = (int(sample_hps.prompt_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens

  x = load_prompts(audio_files, duration, hps)
  zs = top_prior.encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'continue':
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zs = [z.cuda() for z in data['zs']]
  zs = _sample(zs, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
elif sample_hps.mode == 'cutcontinue':
  print('-------CUT INIT--------')
  lecutlen = (int(lecut*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  print(lecutlen)
  data = t.load(codes_file, map_location='cpu')
  zabaca = [z.cuda() for z in data['zs']]
  print(zabaca)
  assert zabaca[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"
  priorsz = [top_prior] * 3
  top_raw_to_tokens = priorsz[-1].raw_to_tokens
  assert lecutlen % top_raw_to_tokens == 0, f"Cut-off duration {lecutlen} not an exact multiple of top_raw_to_tokens"
  assert lecutlen//top_raw_to_tokens <= zabaca[-1].shape[1], f"Cut-off tokens {lecutlen//priorsz[-1].raw_to_tokens} longer than tokens {zs[-1].shape[1]} in saved codes"
  zabaca = [z[:,:lecutlen//prior.raw_to_tokens] for z, prior in zip(zabaca, priorsz)]
  hps.sample_length = lecutlen
  print(zabaca)
  zs = _sample(zabaca, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
  del data
  print('-------CUT OK--------')
  hps.sample_length = (int(sample_length_in_seconds*hps.sr)//top_prior.raw_to_tokens)*top_prior.raw_to_tokens
  data = t.load(sample_hps.codes_file, map_location='cpu')
  zibica = [z.cuda() for z in data['zs']]
  zubu = zibica[:]
  if transpose != [0,1,2]:
    zubu[2][0] = zibica[:][2][transpose[0]];zubu[2][1] = zibica[:][2][transpose[1]];zubu[2][2] = zibica[:][2][transpose[2]]
    zubu[1][0] = zibica[:][1][transpose[0]];zubu[1][1] = zibica[:][1][transpose[1]];zubu[1][2] = zibica[:][1][transpose[2]]
    zubu[0][0] = zibica[:][0][transpose[0]];zubu[0][1] = zibica[:][0][transpose[1]];zubu[0][2] = zibica[:][0][transpose[2]]
  zubu = _sample(zubu, labels, sampling_kwargs, [None, None, top_prior], [2], hps)
  print('-------CONTINUE AFTER CUT OK--------')
  zs = zubu
else:
  raise ValueError(f'Unknown sample mode {sample_hps.mode}.')



print(datetime.now().strftime("%H:%M:%S"))



Cloning into 'jukebox-saveopt'...
remote: Enumerating objects: 1128, done.
remote: Counting objects: 100% (310/310), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 1128 (delta 275), reused 270 (delta 270), pack-reused 818
Receiving objects: 100% (1128/1128), 2.76 MiB | 16.73 MiB/s, done.
Resolving deltas: 100% (623/623), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./jukebox-saveopt


# 2 🥳 Upsample 🥳


In [ ]:
!wget -O /root/.cache/jukebox/models/5b/prior_level_1.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar
!wget -O /root/.cache/jukebox/models/5b/prior_level_0.pth.tar https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar

del top_prior
empty_cache()
top_prior=None

import gc
gc.collect()
os.remove('/root/.cache/jukebox/models/5b/vqvae.pth.tar')



print(datetime.now().strftime("%H:%M:%S"))
del top_prior
empty_cache()
top_prior=None


upsamplers = [make_prior(setup_hparams(prior, dict()), vqvae, 'cpu') for prior in priors[:-1]]
labels[:2] = [prior.labeller.get_batch_labels(metas, 'cuda') for prior in upsamplers]

zs = upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)
print(datetime.now().strftime("%H:%M:%S"))



<font size=60> 🕴️‍♀️ </font>